# 7. 챗봇API

## 7.1 chatbot api?

* 실습한 챗봇엔진서버와 직접통신을 하는 `카카오톡, 네이버톡톡등과 같은 메신저 플랫폼과 챗봇엔진을 사용할 수 있는 챗봇API서버를 구축`해야 한다.
* 챗봇기능을 지원하는 매시전플랫폼과 통신하기 위해서는 `REST API방식으로 챗봇서버를 구현`해야 한다.

## 7.2 파이썬 Flask

* `웹애플리케이션 경량 프레임워크 Flask`를 사용
* Flask는 파이썬 기반의 경령화된 프레임워크

### 7.2.1 Hello Flask

In [ ]:
!mkdir .\hello_flask
!pip show flask

In [ ]:
# %%writefile ./hello_flask/app.py
# 1. "Hello Flask"출력
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return 'Hello Flask!!'

if __name__ == '__main__':
    app.run()

In [ ]:
%%writefile ./hello_flask/app.py
# 2. 동적으로 변수처리 방법
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def hello():
    return 'Hello Flask!!'
    # return render_template("hello.html", name="홍길동")

@app.route('/info/<name>') # http://localhost:5000/infor/홍길동
def get_name(name):
    return f"Hello {name}"

@app.route('/user/<int:id>')
def get_user(id):
    return f"User Id = {id}"

# http://127.0.0.1:5000/JSON/1000/자장면 1개 주문할게요
@app.route('/JSON/<int:dest_id>/<message>')
def send_message(dest_id, message):
    json = {
        "bot_id": dest_id,
        "message": message
    }
    return json

if __name__ == '__main__':
    app.run()

### 7.2.2 기본적인 Rest API 서비스 구현

In [ ]:
!mkdir .\basic_restapi

In [3]:
%%writefile ./basic_restapi/app.py
# 3. 기본적인 Rest Api 서버 구현하기
from flask import Flask, request, jsonify

app = Flask(__name__)

# 서버리소스
resource = []

# 1) 사용자정보조회
@app.route('/user/<int:user_id>', methods=['GET'])
def get_user(user_id):
    
    for user in resource:
        if user['user_id'] is user_id:
            return jsonify(user)
        
    return jsonify(None)
    
# 2) 사용자추가
@app.route('/user', methods=['POST'])
def add_user():
    user = request.get_json()
    resource.append(user)
    return jsonify(resource)
    
if __name__ == '__main__':
    app.run()

Overwriting ./basic_restapi/app.py


## 7.3 챗봇API구현

##### 발화자질문
```json
{
  "query":"오늘 자장면 주문할게요"
}
```

##### 질의답변
```json
{
    "Answer": "자장면 주문 처리 완료되었습니다. \n주문해주셔서 감사합니다.",
    "AnswerImageUrl": null,
    "Intent": "주문",
    "NER": "[('오늘', 'B_DT'), ('자장면', 'B_FOOD'), ('주문', 'O')]",
    "Query": "오늘 자장면 주문할게요"
}
```

In [4]:
!mkdir .\chatbot_api

In [13]:
%pwd

'd:\\lec\\04.python'

In [21]:
%%writefile ./chatbot_api/app.py
# chatbot rest api 서버 구현하기
from flask import Flask, request, jsonify, abort, render_template
import socket
import json

# 챗봇엔진서버접속정보
host = "127.0.0.1"  # chatbot api server ip address
port = 5050         # chatbot api server port no

# flask app
app = Flask(__name__)

# ./chatbot_api/templates/hell.html
@app.route('/', methods=["GET"])
def index():
    return render_template("hello.html")
    # return "Hello!!!"
    
# 1) chatbot server와 통신
def get_answer_from_engine(bottype, query):
    # chatbot server와 연결
    mySocket = socket.socket()
    mySocket.connect((host, port))
    
    # chatbot server에 request(query)
    json_data = {
        'Query': query,
        'BotType': bottype
    }
    
    # chatbot server에 query 전송
    message = json.dumps(json_data)
    mySocket.send(message.encode()) 
    
    # chatbot server에서 답변 리턴
    data = mySocket.recv(2048).decode()
    ret_data = json.loads(data)
    
    # chatbot server에 연결된 socket 자원 해제
    mySocket.close()
    
    return ret_data

# 2) chatbot엔진 query 전송 API
# http://localhost:5000/query/TEST
@app.route('/query/<bot_type>', methods=["POST"]) 
def query(bot_type):
    
    body = request.get_json()
    
    try:
        if bot_type == 'TEST':
            # chatbot api test
            ret = get_answer_from_engine(bottype=bot_type, query=body['query'])
            return jsonify(ret)
            
        elif bot_type == "KAKAO": # 카카오톡 skill 처리
            pass
        elif bot_type == "NAVER": # 네이버톡톡 Web Hook 처리
            pass
        else:
            # 정의되지 않은 bot_type인 경우 404 오류 발생
            abort(404)
    except Exception as e:
        # 예외발생시 500 에러 발생
        abort(500)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)  # web server

Overwriting ./chatbot_api/app.py
